# Warming Up

> Easy and not so easy exercises


## Exercise A

In [ ]:
from itertools import chain, cycle, islice
from collections import deque
from typing import Callable, Iterator, Tuple, List
from collections.abc import Iterable
from numbers import Number
from operator import add

- Write a function `powerOf2(n: int) -> bool` which returns True iff  n is a power of 2
- Restriction: no logarithm, no exp-function allowed

In [ ]:
#collapse
def powerOf2(n: int) -> bool:
    """
    :param n: an integer > = 1
    :return: true if n is a power of two
    """
    return not n & (n - 1)

- Write a function `log2(n: int) -> int` which returns the number of binary digits of n minus one

In [ ]:
#collapse
def log2(n):
    """
    :param n: an integer >= 0
    :return: (number of binary digits of n) - 1
    So: log2(1) = 0, log2(2) = 1, log2(3) = 1
    """
    if n < 1:
        raise ValueError
    result = -1
    while n > 0:
        n >>= 1
        result += 1
    return result

- We consider half-open intervals such as u = [a, b) given as a tuple `(a, b)`.
Write a function `intersection(u: (int, int), v: (int, int)) -> (int, int)`
which returns the intersection of the intervals `u` and `v`.
- Question: How do you manage the empty interval?
- Hint: This is a one-liner

In [ ]:
#collapse
def intersection(u: (int, int), v: (int, int)) -> (int, int):
    """
    :param u: half open interval (u0, u1)
    :param v: half open interval (v0, v1)
    :return: intersection of u and v
    Convention: an interval u is empty iff u[0] >= u[1]
    """
    return max(u[0], v[0]), min(u[1], v[1])

- Write a non-recursivefunction `faculty(n: int) -> long` which returns the faculty of n.

In [ ]:
#collapse
def faculty(n: int) -> int:
    """
    :param n: integer
    :return:  nth-faculty
    standard solution
    """
    if n < 0:
        raise ValueError
    else:
        result = 1
        for i in range(1, n + 1):
            result *= i
        return result


- Write a recursivefunction `faculty1(n: int) -> long` which returns the faculty of n.

In [ ]:
#collapse
def faculty1(n: int) -> int:
    """
    :param n: integer
    :return:  nth-faculty
    recursive solution
    """
    if n < 0:
        raise ValueError
    elif n <= 1:
        return 1
    else:
        return n * faculty1(n - 1)

- Write a function `fibo(n: int) -> int` which returns the n-th fibonacci number

In [ ]:
#collapse
def fibo(n: int) -> int:
    """
    :param n: integer >= 0
    :return: n-th Fibonacci number
    standard solution
    """
    if n < 0:
        raise ValueError
    elif n == 0:
        return 0
    else:
        a, b = 0, 1
        for _ in range(2, n + 1):
            a, b = b, a + b
        return b

- Write a recursive function `fibo(n: int) -> int` which returns the n-th fibonacci number
- How deep is the call stack?

In [ ]:
#collapse
def fibo2(n: int) -> int:
    """
    :param n: integer >= 0
    :return: n-th Fibonacci number
    """
    # recursive programming, cool but slow
    if n < 0:
        raise ValueError
    elif n <= 1:
        return n
    else:
        return fibo2(n - 2) + fibo2(n - 1)

- Write a function `gcd(a: int, b: long) -> long` which returns the greatest common divisor of a and b
- How do you deal with negative or zero arguments?

In [ ]:
#collapse
def gcd(a: int, b: int) -> int:
    """
    :param a: integer
    :param b: integer
    :return: greatest common divisor of a and b
    standard solution
    """
    while b != 0:
        a, b = b, a % b
    return a

- Write a recursive function `gcd(a: int, b: long) -> long` which returns the greatest common divisor of a and b
- How do you deal with negative or zero arguments?

In [ ]:
#collapse
def gcd1(a, b):
    """
    :param a: integer
    :param b: integer
    :return: greatest common divisor of a and b
    recursive solution
    """
    return a if b == 0 \
        else gcd1(b, a % b)

- Write a function `geo(factor: Number, start: Number) -> Iterator` which returns the geometric series starting at `start` with the given `factor`.
  - Hint: Use `fun()`

In [ ]:
#collapse
def geo(factor: Number, start: Number = 1) -> Iterator[Number]:
    """
    :param factor: any number
    :param start: any number, starting value
    :return: the geometric series staring at start
    """
    return fun(lambda x: x * factor, start)

- Write a function `fibo()` which returns the Fibonacci numbers.
  - Hint: Use `fun()`

In [ ]:
#collapse
def fibo() -> Iterator[int]:
    """
    :return: iterator yielding the fibonacci series
    """
    return fun(add, 1, 1)

- Write a class `Fibo()` which can be used as an Iterable and returns the Fibonacci numbers.
  - Hint: Overload `__iter__(self)`

In [ ]:
#collapse
class Fibo(object):
    def __iter__(self):
        return fibo()

- Write a function `alternate() -> Iterator` which alternates between the arithmetic and geometric series.
  - It goes: `ari0, geo0, ar1, geo1, ...`

In [ ]:
#collapse
def alternate() -> Iterator[int]:
    a = ari(1)
    g = geo(2)
    while True:
        yield next(a)
        yield next(g)

- Write a function `hamming(*ps: int) -> Iterator` which produces all multiples of `p0`, `p1`, `p2`, ...
  - So, hamming(2, 3, 5) yields 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, ...
  - Hint: This solution follows Dijkstra: "An exercise attributed to R.W. Hamming":
    - Let `q` be the sequence of multiples produced so far.
    - Then append `min{p*x | x in q, p in ps, p*x > max(q)}` to `q`.
    - The next number to be produced is `min(q)`.

In [ ]:
#collapse
def hamming(*ps: int) -> Iterator[int]:
    """
    :param ps: ps = (p0, p1, p2, ..) contains one or more integers, generally primes
    :return: an iterator yielding all multiples of p0, p1, p2, ..

    This solution follows Dijkstra: "An exercise attributed to R.W. Hamming":
    Let q be the sequence of multiples produced so far.
    Then append min{p*x | x in q, p in ps, p*x > max(q)} to q.
    The next number to be produced is min(q)
    """

    q = [1]  # q contains all numbers produced so far
    while True:
        yield q[-1]
        mq = max(q)
        q.append(min([p * x for p in ps for x in q if p * x > mq]))


## Exercise C

- Write a function `skip_duplicates(t: Iterable) -> Iterator` which skips successive duplicates of `t`

In [ ]:
#collapse
def skip_duplicates(t: Iterable) -> Iterator:
    """"
    :param t: an iterator
    :return: an iterator skipping successive duplicates
    """
    t = iter(t)
    last = next(t)
    yield last

    while True:
        x = next(t)
        if x != last:
            last = x
            yield last

- Write a function `merge(*ts: Iterable) -> Iterator` which merges n non-descending iterators into one.
  - Hint: Keep a dictionary `heads` with key = given iterator `t` and value = last element read (i.e. head of `t`)

In [ ]:
#collapse
def merge(*ts: Iterable) -> Iterator:
    """
    :param ts: a list of n non-descending iterables, n >= 0
    :return: merge of n iterables into one
    This is a weak merge:
    It doesn't stop before the longest iterator is exhausted
    """
    ts = [iter(t) for t in ts]
    heads = {}.fromkeys(ts)  # dictionary of last read entries, initially all None

    while True:
        # get first element of each iterable if there is one
        for t in ts:
            if heads[t] is None:
                h = take(t, 1)  # try to read next element
                heads[t] = h[0] if len(h) > 0 else None

        # get non-exhausted entries
        active_ts = [t for t in ts if heads[t] is not None]
        if not active_ts:  # no active entry left
            return
        else:
            # get non-exhausted entry with minimum value
            min_t = min(active_ts, key=lambda t: heads[t])
            yield heads[min_t]
            heads[min_t] = None  # to be updated on next loop

- Write a recursive function `merge(s, t: Iterable) -> Iterator` which merges two non-descending iterators into one.
  - Hint: The solution follows exactly the list-based solution. Use `chain()`.

In [ ]:
#collapse
def merge1(s, t: Iterable) -> Iterator:
    """
    :param s: a non-descending iterable,
    :param t: a non-descending iterable,
    :return: merge of s and t into one
    This is a weak merge:
    It doesn't stop before the longest iterator is exhausted
    Elegant but inefficient (max recursion depth)
    """
    s, t = iter(s), iter(t)  # get iterator from iterable
    head_s, head_t = take(s, 1), take(t, 1)  # take first element

    if not head_s:
        yield from chain(head_t, t)  # restore first element of t
    elif not head_t:
        yield from chain(head_s, s)  # restore first element of s
    elif head_s[0] <= head_t[0]:
        yield head_s[0]
        yield from merge1(s, chain(head_t, t))
    else:
        yield head_t[0]
        yield from merge1(chain(head_s, s), t)

- Write a function `tee(t: Iterable, n: int) -> Tuple[Iterator]` which forks an iterable into n copies to be iterated on independently.
  - Hints:
    - Keep a list of n deques, one for each fork. This is the buffer.
    - Each iterator is fed from its deque; you have to fill the deques if they are exhausted. This is `itertools.tee`.

In [ ]:
#collapse
def tee1(t: Iterable, n: int = 2) -> Tuple[Iterator]:
    """
    :param t: an iterable
    :param n: number of forks
    :return: n copies of t, to be iterated on independently
    This is a remake of itertools.tee
    """
    t = iter(t)
    buffer = [deque() for _ in range(n)]

    def gen(q: deque) -> Iterator:
        while True:
            if not q:  # when the local deque is empty
                x = next(t)  # fetch a new value and
                for d in buffer:  # load it into buffer
                    d.append(x)
            yield q.popleft()

    return tuple(gen(d) for d in buffer)